In [1]:
import pandas as pd

DATA = '/kaggle/input/leukemia-cancer-risk-prediction-dataset/biased_leukemia_dataset.csv'
df = pd.read_csv(filepath_or_buffer=DATA, index_col=['Patient_ID'])
for column in ['Genetic_Mutation', 'Family_History', 'Smoking_Status', 'Alcohol_Consumption', 'Radiation_Exposure', 'Infection_History', 'Chronic_Illness', 'Immune_Disorders']:
    df[column] = df[column] == 'Yes'
df.head()

,Age,Gender,Country,WBC_Count,RBC_Count,Platelet_Count,Hemoglobin_Level,Bone_Marrow_Blasts,Genetic_Mutation,Family_History,...,Alcohol_Consumption,Radiation_Exposure,Infection_History,BMI,Chronic_Illness,Immune_Disorders,Ethnicity,Socioeconomic_Status,Urban_Rural,Leukemia_Status
Patient_ID,,,,,,,,,,,,,,,,,,,,,
1,52,Male,China,2698,5.36,262493,12.2,72,True,False,...,False,False,False,24.0,False,False,Ethnic_Group_B,Low,Rural,Negative
2,15,Female,China,4857,4.81,277877,11.9,97,True,False,...,False,False,False,28.7,False,False,Ethnic_Group_A,Low,Urban,Positive
3,72,Male,France,9614,5.17,319600,13.4,94,False,True,...,True,False,False,27.7,False,False,Ethnic_Group_B,Low,Urban,Negative
4,61,Male,Brazil,6278,5.41,215200,11.6,50,False,False,...,False,False,False,31.6,False,False,Ethnic_Group_A,Medium,Rural,Negative
5,21,Male,Brazil,8342,4.78,309169,14.3,28,False,False,...,False,False,False,22.3,False,False,Ethnic_Group_B,Low,Rural,Negative


In [2]:
df.nunique()

Age                        89
Gender                      2
Country                    22
WBC_Count               11129
RBC_Count                 390
Platelet_Count          99585
Hemoglobin_Level          164
Bone_Marrow_Blasts        100
Genetic_Mutation            2
Family_History              2
Smoking_Status              2
Alcohol_Consumption         2
Radiation_Exposure          2
Infection_History           2
BMI                       390
Chronic_Illness             2
Immune_Disorders            2
Ethnicity                   3
Socioeconomic_Status        3
Urban_Rural                 2
Leukemia_Status             2
dtype: int64

In [3]:
COLUMNS = [column for column, dtype in df.dtypes.to_dict().items() if str(dtype) in {'bool', 'float64', 'int64'} and column != 'Leukemia_Status']
RANDOM_STATE = 2025
TARGET = 'Leukemia_Status'

In [4]:
df['Leukemia_Status'].value_counts().to_dict()

{'Negative': 121797, 'Positive': 21397}

In [5]:
sample_df = df.sample(n=5000, random_state=RANDOM_STATE)

In [6]:
from sklearn.manifold import TSNE

reducer = TSNE(random_state=RANDOM_STATE)
plot_df = pd.DataFrame(columns=['x', 'y'], data=reducer.fit_transform(X=sample_df[COLUMNS],))
plot_df[TARGET] = sample_df[TARGET].tolist()

In [7]:
from plotly import express
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)
express.scatter(data_frame=plot_df, x='x', y='y', color=TARGET).show(renderer='iframe_connected')